In [3]:
import numpy as np
import os 
os.environ['TF_ENABLE_ONEDNN_OPTS']='0' #to prevent warning message 
import tensorflow as tf 
import tensorflow_datasets as tfds
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
ALL_DATA_DIR = '/media/windows/'
training = ALL_DATA_DIR + 'training/'
validation = ALL_DATA_DIR + 'validation/'
scaled = ALL_DATA_DIR + 'scaled/'
scaled_wo_rotate = ALL_DATA_DIR + "scaled_wo_rotate/"

scaled_wo_rotate_training = scaled_wo_rotate + "training/"
scaled_wo_rotate_validation = scaled_wo_rotate + "validation/"

datFile = training + "shape_predictor_68_face_landmarks.dat"

In [4]:
batch_size = 32 
img_height = 224
img_width = 224 

In [5]:
def preprocess(image, label):
    final_image = keras.applications.xception.preprocess_input(image)
    return final_image, label

In [5]:
validation_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_validation, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=None, seed=42) 

Found 50947 files belonging to 7 classes.


2022-08-25 13:01:54.554115: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-25 13:01:54.563086: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-25 13:01:54.565763: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-25 13:01:54.568918: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [6]:
training_s = tf.keras.utils.image_dataset_from_directory(scaled_wo_rotate_training, labels = 'inferred',
            image_size=(img_height,img_width), batch_size=None, seed=42)

Found 399839 files belonging to 7 classes.


In [7]:
validation_ds = validation_s.map(preprocess).batch(batch_size).prefetch(1)
training_ds = training_s.map(preprocess).batch(batch_size).prefetch(1)

In [6]:
base_model = keras.applications.Xception(input_shape=(224,224,3), include_top = False, weights='imagenet')

2022-08-30 10:19:50.525687: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-30 10:19:50.593106: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-30 10:19:50.595797: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-30 10:19:50.600083: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [7]:
avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(7, activation='softmax')(avg)
model = keras.Model(inputs=base_model.input, outputs=output)

In [10]:
for layer in base_model.layers:
    layer.trainable = False

In [11]:
model.compile(optimizer="Adam",
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

## freezing trainable layers

In [12]:
model.fit(training_ds, validation_data=validation_ds, epochs=3)

Epoch 1/3


2022-08-25 13:02:48.641759: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


12495/12495 [==============================] - 1835s 146ms/step - loss: 1.4592 - accuracy: 0.4372 - val_loss: 1.3922 - val_accuracy: 0.4602
Epoch 2/3
12495/12495 [==============================] - 1766s 141ms/step - loss: 1.3872 - accuracy: 0.4688 - val_loss: 1.3678 - val_accuracy: 0.4766
Epoch 3/3
12495/12495 [==============================] - 1793s 143ms/step - loss: 1.3694 - accuracy: 0.4768 - val_loss: 1.3670 - val_accuracy: 0.4721


## unfreezing trainable layers

In [8]:
for layer in base_model.layers:
    layer.trainable = True

In [9]:
opt = keras.optimizers.Adam(learning_rate=1e-4) #with unfreezing, lessen the learning rate (0.0001)

In [10]:
es = keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True) 
#why? if there is no loss difference between epochs=> stop the fitting 

In [11]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 111, 111, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 111, 111, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                             

 block4_sepconv1_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block4_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block4_sepconv2_act (Activatio  (None, 28, 28, 728)  0          ['block4_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 28, 28, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block4_sepconv2_bn (BatchNorma  (None, 28, 28, 728)  2912       ['block4_sepconv2[0][0]']        
 lization)

 n)                                                                                               
                                                                                                  
 block7_sepconv1 (SeparableConv  (None, 14, 14, 728)  536536     ['block7_sepconv1_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv1_bn (BatchNorma  (None, 14, 14, 728)  2912       ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_sepconv2_act (Activatio  (None, 14, 14, 728)  0          ['block7_sepconv1_bn[0][0]']     
 n)                                                                                               
          

 block9_sepconv3_bn (BatchNorma  (None, 14, 14, 728)  2912       ['block9_sepconv3[0][0]']        
 lization)                                                                                        
                                                                                                  
 add_7 (Add)                    (None, 14, 14, 728)  0           ['block9_sepconv3_bn[0][0]',     
                                                                  'add_6[0][0]']                  
                                                                                                  
 block10_sepconv1_act (Activati  (None, 14, 14, 728)  0          ['add_7[0][0]']                  
 on)                                                                                              
                                                                                                  
 block10_sepconv1 (SeparableCon  (None, 14, 14, 728)  536536     ['block10_sepconv1_act[0][0]']   
 v2D)     

                                                                                                  
 block12_sepconv3_act (Activati  (None, 14, 14, 728)  0          ['block12_sepconv2_bn[0][0]']    
 on)                                                                                              
                                                                                                  
 block12_sepconv3 (SeparableCon  (None, 14, 14, 728)  536536     ['block12_sepconv3_act[0][0]']   
 v2D)                                                                                             
                                                                                                  
 block12_sepconv3_bn (BatchNorm  (None, 14, 14, 728)  2912       ['block12_sepconv3[0][0]']       
 alization)                                                                                       
                                                                                                  
 add_10 (A

In [16]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(training_ds, validation_data=validation_ds, epochs=5, callbacks=[es])
model.save('0825_model.h5')

Epoch 1/5
12495/12495 [==============================] - 7581s 606ms/step - loss: 0.7383 - accuracy: 0.7267 - val_loss: 0.6317 - val_accuracy: 0.7691
Epoch 2/5
12495/12495 [==============================] - 7602s 608ms/step - loss: 0.4903 - accuracy: 0.8209 - val_loss: 0.5502 - val_accuracy: 0.8100
Epoch 3/5
12495/12495 [==============================] - 7526s 602ms/step - loss: 0.3434 - accuracy: 0.8754 - val_loss: 0.6202 - val_accuracy: 0.8089
Epoch 4/5
12495/12495 [==============================] - 7560s 605ms/step - loss: 0.2523 - accuracy: 0.9072 - val_loss: 0.6684 - val_accuracy: 0.8157


In [64]:
model.evaluate(validation_ds)

1593/1593 [==============================] - 208s 130ms/step - loss: 0.5502 - accuracy: 0.8100


[0.5501794815063477, 0.809998631477356]

In [72]:
Error = {}
j=0
for image, label in validation_ds:
    A = image
    B = label
    Pr = np.round(model.predict(A), 2)
    Tr = np.array(B)
    for i in range(len(B)):
        if np.argmax(Pr[i]) != Tr[i]:
            Error[j] = str(Pr[i]) + str(Tr[i])
            j = j+1

1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 41ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 32ms/step


1/1 [==============================] - 0s 24ms/step


In [73]:
C = model.predict(validation_ds)

1593/1593 [==============================] - 207s 130ms/step


In [74]:
D = np.round(C, 2)

In [57]:
Error = {}
Tr = np.array(B)
for i in range(len(B)):
    if np.argmax(Pr[i]) != Tr[i]:
        Error[i] = str(Pr[i], Tr[i])

In [75]:
D.shape

(50947, 7)

In [77]:
import pandas as pd
E = pd.DataFrame(D)

In [79]:
F = pd.DataFrame.from_dict(Error, orient = 'index')

In [80]:
E.to_csv('0826_prob.csv')
F.to_csv('0826_err.csv')